In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Week 3 — Baselines & Evaluation (BTS On-Time)
- Temporal split: Train(2024), Val(2025-01..03), Test(2025-04)
- Train-only profiles to avoid leakage (route_* / carrier_*)
- Regression: ArrDelay  (MAE/RMSE/R2)
- Classification: ArrDel15 (ROC-AUC/PR-AUC + threshold analysis)
- Clustering: route-level KMeans with silhouette sweep (k=3..8)
"""

import argparse, json
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (mean_absolute_error, mean_squared_error, r2_score,
                             roc_auc_score, average_precision_score,
                             precision_recall_curve, confusion_matrix)
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt


# --------------------------
# CLI (robust to Jupyter args)
# --------------------------
def parse_args(argv=None):
    p = argparse.ArgumentParser(description="Week 3 baselines on Week-2 features")
    p.add_argument("--root", type=str, default="./bts_on_time_data",
                   help="Project root (contains eda/ and parquet/)")
    p.add_argument("--sample", type=str, default="eda/sample_100k_week2_features.parquet",
                   help="Relative path to Week-2 features parquet")
    p.add_argument("--seed", type=int, default=42)
    args, unknown = p.parse_known_args(argv)
    if unknown:
        print(f"[INFO] Ignoring unknown arguments: {unknown}")
    return args


# --------------------------
# Utilities
# --------------------------
PROFILE_COLS = [
    "route_med_arr_delay","route_ontime_rate","route_flights",
    "carrier_med_arr_delay","carrier_ontime_rate","carrier_flights"
]

def safe_carrier_col(df):
    for c in ["Reporting_Airline", "IATA_CODE_Reporting_Airline", "UniqueCarrier"]:
        if c in df.columns:
            return c
    raise KeyError("Carrier column not found")

def metrics_reg(y_true, y_pred):
    return {
        "MAE": float(mean_absolute_error(y_true, y_pred)),
        "RMSE": float(mean_squared_error(y_true, y_pred, squared=False)),
        "R2": float(r2_score(y_true, y_pred)),
    }

def choose_best_threshold(y_true, y_score):
    ps, rs, ts = precision_recall_curve(y_true, y_score)[:3]
    f1s = 2 * (ps * rs) / (ps + rs + 1e-12)
    i = int(np.nanargmax(f1s))
    thr = float(ts[i-1]) if i > 0 else 0.5
    return thr, float(f1s[i]), float(ps[i]), float(rs[i])

def confusion_at(y_true, y_prob, thr):
    pred = (y_prob >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, pred).ravel()
    return {"threshold": float(thr), "TP": int(tp), "FP": int(fp), "TN": int(tn), "FN": int(fn)}

def build_profiles_train(df_train, carrier_col):
    r = (df_train.groupby(["Origin","Dest"], dropna=False)
         .agg(route_med_arr_delay=("ArrDelay","median"),
              route_ontime_rate=("ArrDel15", lambda s: 1 - np.nanmean(pd.to_numeric(s, errors="coerce"))),
              route_flights=("ArrDel15","count"))
         .reset_index())
    c = (df_train.groupby([carrier_col], dropna=False)
         .agg(carrier_med_arr_delay=("ArrDelay","median"),
              carrier_ontime_rate=("ArrDel15", lambda s: 1 - np.nanmean(pd.to_numeric(s, errors="coerce"))),
              carrier_flights=("ArrDel15","count"))
         .reset_index())
    return r, c

def attach_profiles(df, route_prof, carrier_prof, carrier_col):
    df = df.drop(columns=[c for c in PROFILE_COLS if c in df.columns], errors="ignore")
    out = df.merge(route_prof, on=["Origin","Dest"], how="left")
    out = out.merge(carrier_prof, on=[carrier_col], how="left")
    return out

def split_temporal(df):
    train = df[df["Year"] == 2024].copy()
    val   = df[(df["Year"] == 2025) & (df["Month"].isin([1,2,3]))].copy()
    test  = df[(df["Year"] == 2025) & (df["Month"] == 4)].copy()
    return train, val, test

def drop_na_target(df, ycol):
    out = df[pd.notna(df[ycol])].copy()
    dropped = len(df) - len(out)
    if dropped:
        print(f"[CLEAN] Dropped {dropped} rows with NaN target '{ycol}'")
    return out


# --------------------------
# Feature spaces (leakage-safe)
# --------------------------
def setup_feature_spaces(df, carrier_col):
    cat = [carrier_col, "Origin", "Dest", "tod_bin", "season"]
    num = ["Distance", "dow", "is_weekend", "quarter",
           "route_med_arr_delay","route_ontime_rate","route_flights",
           "carrier_med_arr_delay","carrier_ontime_rate","carrier_flights",
           "CRSDepTime_min"]  # scheduled signal

    num_reg = list(num)
    if "DepDelay" in df.columns: num_reg.append("DepDelay")
    if "TaxiOut"  in df.columns: num_reg.append("TaxiOut")

    num_clf = list(num)
    return (num_reg, cat, "ArrDelay"), (num_clf, cat, "ArrDel15")


def build_preprocessor(num_cols, cat_cols):
    # Numeric: median impute -> scale
    num_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])
    # OneHotEncoder API compatibility
    try:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse=False)
    cat_pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("ohe", ohe)
    ])
    return ColumnTransformer(
        transformers=[
            ("num", num_pipe, num_cols),
            ("cat", cat_pipe, cat_cols),
        ],
        remainder="drop"
    )


# --------------------------
# Models
# --------------------------
def regression_pipelines(prep):
    return {
        "linreg": Pipeline([("prep", prep), ("mdl", LinearRegression())]),
        "rf": Pipeline([("prep", prep),
                        ("mdl", RandomForestRegressor(
                            n_estimators=250, random_state=42, n_jobs=-1))]),
    }

def classification_pipelines(prep):
    # Includes Logistic Regression, Random Forest, and Decision Tree baselines
    return {
        "logreg": Pipeline([("prep", prep),
                            ("mdl", LogisticRegression(
                                max_iter=1000, class_weight="balanced"))]),
        "rf": Pipeline([("prep", prep),
                        ("mdl", RandomForestClassifier(
                            n_estimators=300, random_state=42, n_jobs=-1,
                            class_weight="balanced"))]),
        "dt": Pipeline([("prep", prep),
                        ("mdl", DecisionTreeClassifier(
                            max_depth=12,            # readable tree
                            min_samples_leaf=100,    # regularization for imbalance
                            class_weight="balanced",
                            random_state=42))]),
    }


# --------------------------
# Clustering (route-level)
# --------------------------
def route_clustering(df):
    agg = (df.groupby(["Origin","Dest"], dropna=False)
             .agg(n=("ArrDel15","count"),
                  ontime=("ArrDel15", lambda s: 1 - np.nanmean(pd.to_numeric(s, errors="coerce"))),
                  med_arr=("ArrDelay","median"),
                  mean_dist=("Distance","mean"))
             .reset_index())
    agg = agg[agg["n"] >= 50].copy()
    from sklearn.preprocessing import StandardScaler as _SS
    X = _SS().fit_transform(agg[["ontime","med_arr","mean_dist"]].values)

    best_k, best_s, best_model = None, -1.0, None
    for k in range(3, 9):
        km = KMeans(n_clusters=k, n_init=20, random_state=42)
        lab = km.fit_predict(X)
        sil = silhouette_score(X, lab)
        if sil > best_s:
            best_k, best_s, best_model = k, sil, km

    agg["cluster"] = best_model.predict(X)
    summary = (agg.groupby("cluster")
                 .agg(routes=("cluster","count"),
                      avg_support=("n","mean"),
                      med_ontime=("ontime","median"),
                      med_arr_delay=("med_arr","median"),
                      med_distance=("mean_dist","median"))
                 .reset_index()
                 .sort_values("routes", ascending=False))
    return agg, summary, {"k": best_k, "silhouette": float(best_s)}


# --------------------------
# MAIN
# --------------------------
def main(argv=None):
    args = parse_args(argv)
    ROOT = Path(args.root)
    SAMPLE = ROOT / args.sample
    if not SAMPLE.exists():
        raise FileNotFoundError(f"Features parquet not found: {SAMPLE}")

    df = pd.read_parquet(SAMPLE)
    carrier_col = safe_carrier_col(df)

    # sanity checks
    req = ["Year","Month","Origin","Dest","ArrDelay","ArrDel15",
           "Distance","CRSDepTime_min","dow","is_weekend","quarter","season","tod_bin"]
    miss = [c for c in req if c not in df.columns]
    if miss:
        raise KeyError(f"Missing required columns: {miss}")

    # temporal split
    train, val, test = split_temporal(df)
    print(f"[SPLIT] train={len(train)}  val={len(val)}  test={len(test)}")

    # recompute profiles on TRAIN only (and drop any preexisting ones)
    rprof, cprof = build_profiles_train(train, carrier_col)
    train = attach_profiles(train, rprof, cprof, carrier_col)
    val   = attach_profiles(val,   rprof, cprof, carrier_col)
    test  = attach_profiles(test,  rprof, cprof, carrier_col)

    # drop rows with NaN targets for regression
    train = drop_na_target(train, "ArrDelay")
    val   = drop_na_target(val,   "ArrDelay")
    test  = drop_na_target(test,  "ArrDelay")

    # feature spaces
    (Xr_num, Xr_cat, y_reg), (Xc_num, Xc_cat, y_clf) = setup_feature_spaces(train, carrier_col)

    # ================= REGRESSION =================
    reg_prep = build_preprocessor(Xr_num, Xr_cat)
    reg_models = regression_pipelines(reg_prep)
    reg_results = {}

    for name, pipe in reg_models.items():
        pipe.fit(train[Xr_num + Xr_cat], train[y_reg])

        def eval_split(s):
            pred = pipe.predict(s[Xr_num + Xr_cat])
            return metrics_reg(s[y_reg], pred), pred

        m_val, _ = eval_split(val)
        m_test, yhat_test = eval_split(test)
        reg_results[name] = {"val": m_val, "test": m_test}

        out_pred = ROOT / f"eda/week3_reg_pred_{name}.csv"
        pd.DataFrame({
            "FlightDate": test.get("FlightDate"),
            "Origin": test["Origin"], "Dest": test["Dest"],
            carrier_col: test[carrier_col],
            "y_true": test[y_reg], "y_pred": yhat_test
        }).to_csv(out_pred, index=False)
        print(f"[REG] {name}  val={m_val}  test={m_test}  -> {out_pred}")

    # ================ CLASSIFICATION ===============
    for s in (train, val, test):
        s[y_clf] = pd.to_numeric(s[y_clf], errors="coerce").fillna(0).astype(int)

    clf_prep = build_preprocessor(Xc_num, Xc_cat)
    clf_models = classification_pipelines(clf_prep)
    clf_results = {}

    for name, pipe in clf_models.items():
        pipe.fit(train[Xc_num + Xc_cat], train[y_clf])

        def eval_split(s):
            prob = pipe.predict_proba(s[Xc_num + Xc_cat])[:,1]
            roc = float(roc_auc_score(s[y_clf], prob))
            pr  = float(average_precision_score(s[y_clf], prob))
            thr, f1b, pb, rb = choose_best_threshold(s[y_clf], prob)
            cm_def  = confusion_at(s[y_clf], prob, 0.5)
            cm_best = confusion_at(s[y_clf], prob, thr)
            return {"ROC_AUC": roc, "PR_AUC": pr,
                    "best_thresh": thr, "best_F1": f1b,
                    "prec_at_best": pb, "rec_at_best": rb,
                    "cm@0.5": cm_def, "cm@best": cm_best}, prob

        m_val, _ = eval_split(val)
        m_test, p_test = eval_split(test)
        clf_results[name] = {"val": m_val, "test": m_test}

        out_prob = ROOT / f"eda/week3_clf_prob_{name}.csv"
        pd.DataFrame({
            "FlightDate": test.get("FlightDate"),
            "Origin": test["Origin"], "Dest": test["Dest"],
            carrier_col: test[carrier_col],
            "y_true": test[y_clf], "y_prob": p_test
        }).to_csv(out_prob, index=False)
        print(f"[CLF] {name}  val={m_val}  test={m_test}  -> {out_prob}")

        # PR curve
        ps, rs, _ = precision_recall_curve(test[y_clf], p_test)
        plt.figure()
        plt.plot(rs, ps)
        plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR Curve (test) — {name}")
        plt.tight_layout()
        fig_path = ROOT / f"eda/week3_clf_pr_{name}.png"
        plt.savefig(fig_path, dpi=120); plt.close()
        print(f"[CLF] PR curve saved -> {fig_path}")

    # ================= CLUSTERING ==================
    routes, cluster_summary, best = route_clustering(df)
    out_routes  = ROOT / "eda/week3_route_clusters.csv"
    out_summary = ROOT / "eda/week3_route_cluster_summary.csv"
    routes.to_csv(out_routes, index=False)
    cluster_summary.to_csv(out_summary, index=False)
    print(f"[CLUSTER] best_k={best['k']}, silhouette={best['silhouette']:.4f}  -> {out_routes} / {out_summary}")

    # ================= METRICS JSON ===============
    summary = {"regression": reg_results,
               "classification": clf_results,
               "clustering": best}
    out_json = ROOT / "eda/week3_metrics.json"
    with open(out_json, "w") as f:
        json.dump(summary, f, indent=2)
    print(f"[DONE] metrics json -> {out_json}")


if __name__ == "__main__":
    import sys
    main(sys.argv[1:])

[INFO] Ignoring unknown arguments: ['--f=/Users/guohaoyang/Library/Jupyter/runtime/kernel-v381a5ca67ad28d9522eecfee939d36f10b14a52c3.json']
[SPLIT] train=67485  val=24163  test=8352
[CLEAN] Dropped 855 rows with NaN target 'ArrDelay'
[CLEAN] Dropped 469 rows with NaN target 'ArrDelay'
[CLEAN] Dropped 84 rows with NaN target 'ArrDelay'
[REG] linreg  val={'MAE': 18635481.848606516, 'RMSE': 676305820.8841715, 'R2': -137667681320031.4}  test={'MAE': 9124250.80308588, 'RMSE': 479162648.70245594, 'R2': -83527695637906.27}  -> bts_on_time_data/eda/week3_reg_pred_linreg.csv
[REG] rf  val={'MAE': 8.864394871114648, 'RMSE': 12.382560257019293, 'R2': 0.9538505374962416}  test={'MAE': 8.341302467343976, 'RMSE': 11.795224227671387, 'R2': 0.9493852624920154}  -> bts_on_time_data/eda/week3_reg_pred_rf.csv
[CLF] logreg  val={'ROC_AUC': 0.5605660748907593, 'PR_AUC': 0.2307595299425911, 'best_thresh': 0.3439192131260163, 'best_F1': 0.33671343242082863, 'prec_at_best': 0.2152880573154124, 'rec_at_best': 

In [5]:
from pathlib import Path
import json, pandas as pd
ROOT = Path("./bts_on_time_data")

with open(ROOT/"eda/week3_metrics.json") as f:
    M = json.load(f)

# 1) Regression scoreboard
rows = []
for mdl, res in M["regression"].items():
    for split in ("val","test"):
        r = res[split]
        rows.append({"Model": mdl, "Split": split,
                     "MAE": r["MAE"], "RMSE": r["RMSE"], "R2": r["R2"]})
reg_df = pd.DataFrame(rows).sort_values(["Split","Model"])
print("\n=== Regression metrics ===")
print(reg_df.to_string(index=False, formatters={"MAE":"{:,.3f}".format,
                                                "RMSE":"{:,.3f}".format,
                                                "R2":"{:.4f}".format}))

# 2) Classification scoreboard
rows = []
for mdl, res in M["classification"].items():
    for split in ("val","test"):
        r = res[split]
        rows.append({"Model": mdl, "Split": split,
                     "ROC_AUC": r["ROC_AUC"], "PR_AUC": r["PR_AUC"],
                     "BestThresh": r["best_thresh"], "BestF1": r["best_F1"],
                     "Prec@Best": r["prec_at_best"], "Rec@Best": r["rec_at_best"]})
clf_df = pd.DataFrame(rows).sort_values(["Split","Model"])
print("\n=== Classification metrics ===")
print(clf_df.to_string(index=False, formatters={"ROC_AUC":"{:.3f}".format,
                                                "PR_AUC":"{:.3f}".format,
                                                "BestThresh":"{:.3f}".format,
                                                "BestF1":"{:.3f}".format,
                                                "Prec@Best":"{:.3f}".format,
                                                "Rec@Best":"{:.3f}".format}))

# 3) Confusion matrices @0.5 and @best (test only)
print("\n=== Test confusion matrices ===")
for mdl in ("logreg","rf","dt"):
    r = M["classification"][mdl]["test"]
    print(f"\n[{mdl}] @0.5  -> {r['cm@0.5']}")
    print(f"[{mdl}] @best -> {r['cm@best']}  (thr={r['best_thresh']:.3f})")

# 4) Positive rates
feats = pd.read_parquet(ROOT/"eda/sample_100k_week2_features.parquet")
pos = lambda df: (pd.to_numeric(df["ArrDel15"], errors="coerce")==1).mean()
prt = pos(feats[feats["Year"]==2024])
prv = pos(feats[(feats["Year"]==2025)&(feats["Month"].isin([1,2,3]))])
prs = pos(feats[(feats["Year"]==2025)&(feats["Month"]==4)])
print("\n=== Positive rate (ArrDel15=1) ===")
print(f"train={prt:.3f}  val={prv:.3f}  test={prs:.3f}")

# 5) Clusters summary + representatives
clusters = pd.read_csv(ROOT/"eda/week3_route_clusters.csv")
summary  = pd.read_csv(ROOT/"eda/week3_route_cluster_summary.csv")
print("\n=== Cluster summary ===")
print(summary.to_string(index=False))
reproutes = (clusters.sort_values(["cluster","n"], ascending=[True,False])
                     .groupby("cluster").head(5)[["cluster","Origin","Dest","n","ontime","med_arr","mean_dist"]])
print("\n=== Representative routes per cluster (top-5) ===")
print(reproutes.to_string(index=False))


=== Regression metrics ===
 Model Split            MAE            RMSE                    R2
linreg  test  9,124,250.803 479,162,648.702  -83527695637906.2656
    rf  test          8.341          11.795                0.9494
linreg   val 18,635,481.849 676,305,820.884 -137667681320031.4062
    rf   val          8.864          12.383                0.9539

=== Classification metrics ===
 Model Split ROC_AUC PR_AUC BestThresh BestF1 Prec@Best Rec@Best
    dt  test   0.577  0.238      0.356  0.349     0.226    0.763
logreg  test   0.582  0.241      0.372  0.347     0.225    0.756
    rf  test   0.585  0.239      0.106  0.348     0.227    0.752
    dt   val   0.560  0.233      0.393  0.334     0.220    0.694
logreg   val   0.561  0.231      0.344  0.337     0.215    0.772
    rf   val   0.565  0.236      0.045  0.338     0.208    0.896

=== Test confusion matrices ===

[logreg] @0.5  -> {'threshold': 0.5, 'TP': 890, 'FP': 2751, 'TN': 3885, 'FN': 742}
[logreg] @best -> {'threshold': 0.3718